In [1]:
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from sklearn import linear_model, datasets

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((15*21,3), np.float32)
objp[:,:2] = np.mgrid[0:21,0:15].T.reshape(-1,2)

In [3]:
objp.shape

(315, 3)

In [6]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('a.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (21,15),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (21,15), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [5]:
print(corners.shape)

NameError: name 'corners' is not defined

In [ ]:
imgp = np.ones((3,315))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,315))
imgp[:2,:] = imgp1
print(imgp.shape)

objp = objp.T
print(objp.shape)
obj = np.ones((4,315))
obj[:3,:] = objp

In [ ]:
obj

In [ ]:
imgp

In [87]:
real = [[1,1],[2,3],[4,5]]
real = np.array(real)
sensed = [[100,200],[200,360],[400,600]]
sensed = np.array(sensed)

In [84]:
def get_normalisation_matrix(flattened_corners):
#     end = timer()

    avg_x = flattened_corners[:, 0].mean()
    avg_y = flattened_corners[:, 1].mean()

    s_x = np.sqrt(2 / flattened_corners[:,0].std())
    s_y = np.sqrt(2 / flattened_corners[:,1].std())

#     end("get_normalization_matrix")
    return np.matrix([
        [s_x,   0,   -s_x * avg_x],
        [0,   s_y,   -s_y * avg_y],
        [0,     0,              1]
    ])

In [89]:
first_normalisation_matrix = get_normalisation_matrix(real)
first_normalisation_matrix

matrix([[ 1.26632044,  0.        , -2.95474769],
        [ 0.        ,  1.10668192, -3.32004576],
        [ 0.        ,  0.        ,  1.        ]])

In [90]:
second_normalisation_matrix = get_normalisation_matrix(sensed)
second_normalisation_matrix

matrix([[  0.12663204,   0.        , -29.54747688],
        [  0.        ,   0.11030234, -42.65023879],
        [  0.        ,   0.        ,   1.        ]])

In [91]:
real

array([[1, 1],
       [2, 3],
       [4, 5]])

In [103]:
fnm = np.ones((3,3))
fnm[:,:2] = real
print(fnm)
snm = np.ones((3,3))
snm[:,:2] = sensed
print(snm)

[[1. 1. 1.]
 [2. 3. 1.]
 [4. 5. 1.]]
[[100. 200.   1.]
 [200. 360.   1.]
 [400. 600.   1.]]


In [112]:
pr_1 = np.dot(first_normalisation_matrix,fnm)
pr_2 = np.dot(second_normalisation_matrix,snm)
print(pr_1)
print(pr_2)

[[-10.55267032 -13.507418    -1.68842725]
 [-11.0668192  -13.28018304  -2.21336384]
 [  4.           5.           1.        ]]
[[-1.18063275e+04 -1.77031597e+04 -2.94208448e+01]
 [-1.70380350e+04 -2.55504344e+04 -4.25399364e+01]
 [ 4.00000000e+02  6.00000000e+02  1.00000000e+00]]


In [119]:
pr_1.item(1)

-13.507418004398486

In [113]:
M=[]

M.append(np.array([pr_1.item(0), pr_1.item(1), 1,0, 0, 0,-pr_1.item(0)*pr_2.item(0), -pr_1.item(1)*pr_2.item(0), -pr_2.item(0)]))

M.append(np.array([0, 0, 0, pr_1.item(0), pr_1.item(1),1, -pr_1.item(0)*pr_2.item(1), -pr_1.item(1)*pr_2.item(1), -pr_2.item(1)]))

In [114]:
M

[array([-1.05526703e+01, -1.35074180e+01,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.24588282e+05, -1.59473001e+05,
         1.18063275e+04]),
 array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.05526703e+01,
        -1.35074180e+01,  1.00000000e+00, -1.86815608e+05, -2.39123978e+05,
         1.77031597e+04])]

In [117]:
U, S, Vh = np.linalg.svd(np.array(M).reshape((2, 9)))

L = Vh[-1]

H = L.reshape(3, 3)

denormalised = np.dot( np.dot (np.linalg.inv(first_normalisation_matrix),H ), second_normalisation_matrix)

In [118]:
denormalised / denormalised[-1, -1]

matrix([[-7.68193772e-03, -4.56451220e-03,  2.40308098e+00],
        [-7.09658972e-03, -7.91226426e-03,  3.23185954e+00],
        [-2.24934303e-03, -2.50788014e-03,  1.00000000e+00]])

In [ ]:
def estimate_homography(first, second):
    end = timer()

    first_normalisation_matrix = get_normalisation_matrix(first)
    second_normalisation_matrix = get_normalisation_matrix(second)

    M = []

    for j in range(0, first.size):
        homogeneous_first = np.array([
            first[j][0],
            first[j][1],
            1
        ])

        homogeneous_second = np.array([
            second[j][0],
            second[j][1],
            1
        ])

        pr_1 = np.dot(first_normalisation_matrix, homogeneous_first)

        pr_2 = np.dot(second_normalisation_matrix, homogeneous_second)

        M.append(np.array([
            pr_1.item(0), pr_1.item(1), 1,
            0, 0, 0,
            -pr_1.item(0)*pr_2.item(0), -pr_1.item(1)*pr_2.item(0), -pr_2.item(0)
        ]))

        M.append(np.array([
            0, 0, 0, pr_1.item(0), pr_1.item(1),
            1, -pr_1.item(0)*pr_2.item(1), -pr_1.item(1)*pr_2.item(1), -pr_2.item(1)
        ]))

    U, S, Vh = np.linalg.svd(np.array(M).reshape((512, 9)))

    L = Vh[-1]

    H = L.reshape(3, 3)

    denormalised = np.dot(
        np.dot(
            np.linalg.inv(first_normalisation_matrix),
            H
        ),
        second_normalisation_matrix
    )

    end("estimate_homography")
    return denormalised / denormalised[-1, -1]


In [83]:
def compute_homography(real, sensed):
    end = timer()

    real = data['real']

    refined_homographies = []

    for i in range(0, len(data['sensed'])):
        sensed = data['sensed'][i]
        estimated = estimate_homography(real, sensed)
#         end = timer()
#         refined = refine_homography(estimated, sensed, real)
#         refined = refined / refined[-1]
#         end("refine_homography")
#         refined_homographies.append(estimated)

#     end("compute_homography")
#     return np.array(refined_homographies)
    return 0

In [3]:
from steps.parser import parse_data
from steps.dlt import compute_homography
from steps.intrinsics import get_camera_intrinsics
from steps.extrinsics import get_camera_extrinsics
from steps.distortion import estimate_lens_distortion
from utils.timer import timer


def calibrate():
    data = parse_data()

    end = timer()
    homographies = compute_homography(data)
    end("Homography Estimation")
    print("homographies")
    print(homographies)

    end = timer()
    intrinsics = get_camera_intrinsics(homographies)
    end("Intrinsics")

    print("intrinsics")
    print(intrinsics)

    end = timer()
    extrinsics = get_camera_extrinsics(intrinsics, homographies)
    end("Extrinsics")

    print("extrinsics")
    print(extrinsics)

    end = timer()
    distortion = estimate_lens_distortion(
        intrinsics,
        extrinsics,
        data["real"],
        data["sensed"]
    )
    end("Distortion")

    return 0

# calibrate()

In [20]:
data = parse_data()
data["real"].shape
len(data["sensed"][0][0])

2

In [22]:
def parse_data(basepath="data/corners_", ext=".dat"):

    sensed = []
    for i in range(1, 6):
        sensed.append(np.loadtxt(basepath + str(i) + ext).reshape((256, 2)))

    return {
        'real': np.loadtxt(basepath + "real" + ext).reshape((256, 2)),
        'sensed': sensed
    }

In [23]:
def compute_homography(data):
    end = timer()

    real = data['real']

    refined_homographies = []

    for i in range(0, len(data['sensed'])):
        sensed = data['sensed'][i]
        estimated = estimate_homography(real, sensed)
        end = timer()
        refined = refine_homography(estimated, sensed, real)
        refined = refined / refined[-1]
        end("refine_homography")
        refined_homographies.append(estimated)

    end("compute_homography")
    return np.array(refined_homographies)

In [24]:
import numpy as np
from scipy import optimize as opt
from utils.timer import timer


def get_normalisation_matrix(flattened_corners):
    end = timer()

    avg_x = flattened_corners[:, 0].mean()
    avg_y = flattened_corners[:, 1].mean()

    s_x = np.sqrt(2 / flattened_corners[0].std())
    s_y = np.sqrt(2 / flattened_corners[1].std())

    end("get_normalization_matrix")
    return np.matrix([
        [s_x,   0,   -s_x * avg_x],
        [0,   s_y,   -s_y * avg_y],
        [0,     0,              1]
    ])


def estimate_homography(first, second):
    end = timer()

    first_normalisation_matrix = get_normalisation_matrix(first)
    second_normalisation_matrix = get_normalisation_matrix(second)

    M = []

    for j in range(0, int(first.size / 2)):
        homogeneous_first = np.array([
            first[j][0],
            first[j][1],
            1
        ])

        homogeneous_second = np.array([
            second[j][0],
            second[j][1],
            1
        ])

        pr_1 = np.dot(first_normalisation_matrix, homogeneous_first)

        pr_2 = np.dot(second_normalisation_matrix, homogeneous_second)

        M.append(np.array([
            pr_1.item(0), pr_1.item(1), 1,
            0, 0, 0,
            -pr_1.item(0)*pr_2.item(0), -pr_1.item(1)*pr_2.item(0), -pr_2.item(0)
        ]))

        M.append(np.array([
            0, 0, 0, pr_1.item(0), pr_1.item(1),
            1, -pr_1.item(0)*pr_2.item(1), -pr_1.item(1)*pr_2.item(1), -pr_2.item(1)
        ]))

    U, S, Vh = np.linalg.svd(np.array(M).reshape((512, 9)))

    L = Vh[-1]

    H = L.reshape(3, 3)

    denormalised = np.dot(
        np.dot(
            np.linalg.inv(first_normalisation_matrix),
            H
        ),
        second_normalisation_matrix
    )

    end("estimate_homography")
    return denormalised / denormalised[-1, -1]


def cost(homography, data):
    [sensed, real] = data

    Y = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        w = homography[6] * x + homography[7] * y + homography[8]

        M = np.array([
            [homography[0], homography[1], homography[2]],
            [homography[3], homography[4], homography[5]]
        ])

        homog = np.transpose(np.array([x, y, 1]))
        [u, v] = (1/w) * np.dot(M, homog)

        Y.append(u)
        Y.append(v)

    return np.array(Y)


def jac(homography, data):
    [sensed, real] = data

    J = []

    for i in range(0, sensed.size / 2):
        x = sensed[i][0]
        y = sensed[i][1]

        s_x = homography[0] * x + homography[1] * y + homography[2]
        s_y = homography[3] * x + homography[4] * y + homography[5]
        w = homography[6] * x + homography[7] * y + homography[8]

        J.append(
            np.array([
                x / w, y / w, 1/w,
                0, 0, 0,
                (-s_x * x) / (w*w), (-s_x * y) / (w*w), -s_x / (w*w)
            ])
        )

        J.append(
            np.array([
                0, 0, 0,
                x / w, y / w, 1 / w,
                (-s_y * x) / (w*w), (-s_y * y) / (w*w), -s_y / (w*w)
            ])
        )

    return np.array(J)


def refine_homography(homography, sensed, real):
    return opt.root(
        cost,
        homography,
        jac=jac,
        args=[sensed, real],
        method='lm'
    ).x
